In [393]:
import pandas as pd
import re
from geopy.geocoders import Nominatim
import re
import nltk
from nltk.corpus import stopwords

In [94]:
geolocalizador = Nominatim(user_agent="M")

In [78]:
#df_fever= pd.read_csv('../../data/data_inicial/FEVER.csv')
#df_fever_detalle= pd.read_csv('../../data/data_inicial/FEVER_DETALLE.csv')
#df_groupon= pd.read_csv('../../data/data_inicial/GROUPON.csv')
#df_groupon_detalle= pd.read_csv('../../data/data_inicial/GROUPON_DETALLE.csv')
#df_tripadvisor= pd.read_json('../1_Extraccion/tutorial/TRIPADVISOR_DETALLE.json')

## Limpieza Fever

In [269]:
df_fever= pd.read_csv('../../data/data_inicial/FEVER.csv')
df_fever_detalle= pd.read_csv('../../data/data_inicial/FEVER_DETALLE.csv')

In [221]:
df_fever.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 912 entries, 0 to 911
Data columns (total 7 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   categoria  912 non-null    object 
 1   nombre     912 non-null    object 
 2   url        912 non-null    object 
 3   precio     886 non-null    object 
 4   fecha      812 non-null    object 
 5   rating     552 non-null    float64
 6   sku        912 non-null    int32  
dtypes: float64(1), int32(1), object(5)
memory usage: 46.4+ KB


In [208]:
df_fever_detalle.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 481 entries, 0 to 480
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   descripcion  481 non-null    object 
 1   sku          481 non-null    int64  
 2   imagen       481 non-null    object 
 3   latitud      481 non-null    float64
 4   longitud     481 non-null    float64
 5   lugar        481 non-null    object 
dtypes: float64(2), int64(1), object(3)
memory usage: 22.7+ KB


In [224]:
df_fever.head(1)

,categoria,nombre,url,precio,fecha,rating,sku
0,conciertos-festivales,Candlelight Orquesta: lo mejor de Joe Hisaishi,https://feverup.com/m/127269,"Desde\n15,00 €",20 abr,NaN,127269


In [230]:
df_fever_all.head(4)


,categoria,nombre,url,precio,fecha,rating,sku,descripcion,imagen,latitud,longitud,lugar
0,conciertos-festivales,Candlelight Orquesta: lo mejor de Joe Hisaishi,https://feverup.com/m/127269,"Desde\n15,00 €",20 abr,NaN,127269,elight son los conciertos a la luz de las vel...,https://applications-media.feverup.com/image/u...,40.420915,-3.718800,Gran Teatro CAIXABANK Príncipe Pío
1,conciertos-festivales,Candlelight Summer 2023 - Lista de espera,https://feverup.com/m/121393,NaN,NaN,NaN,121393,Candlelight Summer 2023 - Lista de espera,NaN,NaN,NaN,NaN
2,conciertos-festivales,Rockin'1000: Una banda de mil músicos en directo,https://feverup.com/m/125023,"Desde\n39,50 €",03 jun,NaN,125023,¿Quieres vivir el concierto en directo de la...,https://applications-media.feverup.com/image/u...,40.436913,-3.599382,Cívitas Metropolitano
3,conciertos-festivales,Candlelight: Requiem de Mozart en San Antonio ...,https://feverup.com/m/128037,"Desde\n15,00 €",07 may,NaN,128037,elight trae la magia de una experiencia music...,https://applications-media.feverup.com/image/u...,40.422359,-3.703813,Iglesia de San Antonio de los Alemanes


In [270]:
#FUNCION EXTRACCION SKU DE URL MOVER A SRC

def extract_sku(url):
    match = re.search(r'\d+$', url)
    if match:
        return match.group()
    else:
        return None

In [271]:
#Extraemos el sku de las url para cruzar los 2 df
df_fever['sku'] = df_fever['url'].apply(extract_sku).astype(int)

#Agrupamos los df
df_fever_all = pd.merge(df_fever, df_fever_detalle, on='sku', how='left')

#Gestionaremos los nulos en la columna descripción reemplazándolos por el nombre de la actividad
df_fever_all['descripcion'].fillna(df_fever_all['nombre'], inplace=True)

#Renombrado de columnas
df_fever_all.rename(columns={'categoria': 'subcategoria'}, inplace=True)

## Limpieza Groupon

In [274]:
df_groupon= pd.read_csv('../../data/data_inicial/GROUPON.csv')
df_groupon_detalle= pd.read_csv('../../data/data_inicial/GROUPON_DETALLE.csv')

In [233]:
df_groupon.info()
df_groupon_detalle.info()
df_groupon.head(1)
df_groupon_detalle.head(1)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 98 entries, 0 to 97
Data columns (total 7 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   categoria  98 non-null     object 
 1   nombre     98 non-null     object 
 2   direccion  98 non-null     object 
 3   url        98 non-null     object 
 4   precio     98 non-null     float64
 5   precio_2   0 non-null      float64
 6   rating     61 non-null     float64
dtypes: float64(3), object(4)
memory usage: 5.5+ KB


In [ ]:
#FUNCION LATITUD Y LONGITUD MOVER A SRC

def obtener_lat_long(direccion):
    ubicacion = geolocalizador.geocode(direccion)
    if ubicacion is not None:
        return pd.Series((ubicacion.latitude, ubicacion.longitude))
    else:
        return pd.Series(('Desconocido', 'Desconocido'))

In [275]:
#Eliminamos la columna precio_2 porque no nos aporta información
df_groupon = df_groupon.drop(['precio_2'], axis=1)

#Obetenemos la latitud y longitud de las direcciones de las actividades
df_groupon[['latitud', 'longitud']] = df_groupon['direccion'].apply(obtener_lat_long)

#Agrupamos los df
df_groupon_all = pd.merge(df_groupon, df_groupon_detalle, on='url', how='left')

#Gestionaremos los nulos en la columna descripción reemplazándolos por el nombre de la actividad
df_groupon_all['descripcion'].fillna(df_groupon_all['nombre'], inplace=True)

#Renombrado de columnas
df_groupon_all.rename(columns={'categoria': 'subcategoria'}, inplace=True)

## Limpieza Tripadvisor

In [277]:
df_tripadvisor= pd.read_json('../1_Extraccion/tutorial/TRIPADVISOR_DETALLE.json')

In [158]:
df_tripadvisor.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3233 entries, 0 to 3232
Data columns (total 6 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   name           3233 non-null   object
 1   url            3233 non-null   object
 2   direction      3233 non-null   object
 3   cuisines       3233 non-null   object
 4   priceRangeNum  3233 non-null   object
 5   rating         3233 non-null   object
dtypes: object(6)
memory usage: 151.7+ KB


In [278]:
df_tripadvisor = df_tripadvisor.drop(['province','price','city', 'price_all','latitude','longitude','meals','specialDiets','locationAll','coordinates','positionlink'], axis=1)
df_tripadvisor = df_tripadvisor.dropna()
df_tripadvisor = df_tripadvisor.reset_index(drop=True)

In [154]:
df_tripadvisor_url= pd.DataFrame(df_tripadvisor["url"])
df_tripadvisor_url.to_csv(f'../../data/data_inicial/TRIPADVISOR_URL.csv', header=True, index=False)

In [280]:
#Crear la columna subcategoria
df_tripadvisor['subcategoria'] = 'restaurantes'

#Renombrado de columnas
df_tripadvisor.rename(columns={'cuisines': 'descripcion', 'name': 'nombre'}, inplace=True)

## DF ENTRENAMIENTO

In [371]:
#Creacion de una copia de cada uno de los df
df_entrenamiento_gr= df_groupon_all.copy()
df_entrenamiento_fv= df_fever_all.copy()
df_entrenamiento_tp= df_tripadvisor.copy()

In [372]:
#Eliminacion de las columnas no implicadas en el modelo de recomendacion
df_entrenamiento_gr.drop(df_entrenamiento_gr.columns.difference(['nombre', 'subcategoria', 'descripcion', 'url']), axis=1, inplace=True)
df_entrenamiento_fv.drop(df_entrenamiento_fv.columns.difference(['nombre', 'subcategoria', 'descripcion', 'url']), axis=1, inplace=True)
df_entrenamiento_tp.drop(df_entrenamiento_tp.columns.difference(['nombre', 'subcategoria', 'descripcion', 'url']), axis=1, inplace=True)

#REEMPLAZAR URL POR ID
#df_entrenamiento_gr.drop(df_entrenamiento_gr.columns.difference(['nombre', 'subcategoria', 'descripcion', 'id']), axis=1, inplace=True)
#df_entrenamiento_fv.drop(df_entrenamiento_fv.columns.difference(['nombre', 'subcategoria', 'descripcion', 'id']), axis=1, inplace=True)
#df_entrenamiento_tp.drop(df_entrenamiento_tp.columns.difference(['nombre', 'subcategoria', 'descripcion', 'id']), axis=1, inplace=True)

In [373]:
#Establecer el orden de las columnas
df_entrenamiento_gr = df_entrenamiento_gr[['nombre', 'subcategoria', 'descripcion', 'url']]
df_entrenamiento_fv = df_entrenamiento_fv[['nombre', 'subcategoria', 'descripcion', 'url']]
df_entrenamiento_tp = df_entrenamiento_tp[['nombre', 'subcategoria', 'descripcion', 'url']]

In [374]:
#Concatenado de los df
df_entrenamiento_all = pd.concat([df_entrenamiento_gr, df_entrenamiento_fv, df_entrenamiento_tp], ignore_index=True)

#Generacion de un id a partir de la url (ESTE PASO HAY QUE HACERLO ANTES EN LA UNIFICACION ANTERIOR DE LOS DF)
df_entrenamiento_all['id'], _ = pd.factorize(df_entrenamiento_all['url'], sort=True)

#Añadir la columna categoría (ESTE PASO HAY QUE HACERLO ANTES EN LA UNIFICACION ANTERIOR DE LOS DF)
rel_categoria= pd.read_csv('../../data/data_inicial/CATEGORIA_RELACION.csv')
df_entrenamiento_all = pd.merge(df_entrenamiento_all, rel_categoria, on='subcategoria', how='left')

### Limpieza

In [388]:
#funcion para limpiar descripciones MOVER A SRC

def limpiar_texto(texto):
    # Eliminar números y caracteres especiales
    texto = re.sub('[^a-zA-ZáéíóúüñÁÉÍÓÚÜÑ]|lunes|martes|miércoles|jueves|viernes|sábado|domingo', ' ', texto)
    
    # Convertir el texto a minúsculas
    texto = texto.lower()
    
    # Tokenizar el texto en palabras
    palabras = nltk.tokenize.word_tokenize(texto, language='spanish')
    
    # Eliminar preposiciones, artículos, adverbios y determinantes
    stopwords_sp = set(stopwords.words('spanish'))
    palabras = [palabra for palabra in palabras if palabra not in stopwords_sp]
    
    # Unir las palabras en una cadena de texto nuevamente
    texto_limpio = ' '.join(palabras)
    
    return texto_limpio

In [389]:
#eliminacion de guiones en subcategoría
df_entrenamiento_all['subcategoria'] = df_entrenamiento_all['subcategoria'].apply(lambda x: x.replace('-', ' '))

#limpieza de descripcion
df_entrenamiento_all['descripcion'] = df_entrenamiento_all['descripcion'].apply(lambda x: limpiar_texto(x))

In [394]:
df_entrenamiento_all['descripcion'] = df_entrenamiento_all['descripcion'].apply(lambda x: limpiar_texto(x))

Las opciones incluyen:
Juego de escape room urbano de 2,5 horas aproximadamente para grupos de dos a seis personas,
De martes a jueves o de viernes a domingo.
Sobre el juego:
El juego de aventura ParaPoly no trata solo de ver los monumentos de la ciudad, sino de escuchar lo que te dicen. Se deben buscar las pistas, investiga las señales y resuelve los acertijo. Para ello es necesario un par de amigos, un móvil inteligente, un bolígrafo y el mapa del juego.
La partida no dura más de un par de horas, por lo que es posible hacer un descanso para comer o cenar, o incluso parar por la noche y continuar al día siguiente.
Más informacion sobre el juego: http://madrid.parapark.es/parapoly


In [395]:
df_entrenamiento_all.head(2)

,nombre,subcategoria,descripcion,url,id,categoria
0,Escape room urbano para grupos de 2 a 6 person...,juegos de escape,opciones incluyen juego escape room urbano hor...,https://www.groupon.es/deals/parapark-madrid-2,3783,Entretenimiento
1,Escape game para 4 a 6 personas en una sala a ...,juegos de escape,incluye escape game salas participantes duraci...,https://www.groupon.es/deals/escapology-ilusio...,3759,Entretenimiento


In [385]:
df_entrenamiento_all.to_csv(f'../../data/data_inicial/RECOMENDACION_ALL.csv', header=True, index=False)